In [1]:
import tensorflow as tf
import numpy as np
import PIL
from PIL import Image

@tf.function
def get_rotation_matrix(rad, height: int, width: int):
    bx_s = float(width // 2)
    by_s = float(height // 2)
    # 先垂直反射, 然后左移bx_s, 下移by_s，（bx_s, by_s）移动到了左上原点的地方
    T_s_d1 = tf.stack([
        tf.concat([tf.constant([1.0]), tf.constant([0.0]), tf.constant([-bx_s]) ], axis = 0),
        tf.concat([tf.constant([0.0]), tf.constant([-1.0]), tf.constant([by_s]) ], axis = 0),
        tf.concat([tf.constant([0.0]), tf.constant([0.0]), tf.constant([1.0]) ], axis = 0)
        ], axis = 0)
    # 旋转
    T_d1_d2 = tf.stack([
        tf.concat([tf.cos([rad]), tf.sin([rad]), tf.constant([0.0]) ], axis = 0),
        tf.concat([-tf.sin([rad]), tf.cos([rad]), tf.constant([0.0]) ], axis = 0),
        tf.concat([tf.constant([0.0]), tf.constant([0.0]), tf.constant([1.0]) ], axis = 0)
        ], axis = 0)
    # 先再度y轴反射， 然后右移bx_s，下移by_s
    T_d2_d = tf.stack([
        tf.concat([tf.constant([1.0]), tf.constant([0.0]), tf.constant([bx_s]) ], axis = 0),
        tf.concat([tf.constant([0.0]), tf.constant([-1.0]), tf.constant([by_s]) ], axis = 0),
        tf.concat([tf.constant([0.0]), tf.constant([0.0]), tf.constant([1.0]) ], axis = 0)
        ], axis = 0)
    rotation_mat =  T_d2_d @ T_d1_d2 @ T_s_d1
    return rotation_mat


@tf.function
def warpAffine(image, matrix_inv, width, height):
    old_height, old_width, channel_num = image.shape
    # dst = tf.Variable(tf.zeros((height, width, channel_num), dtype=tf.float32), trainable=False)
    
    i = tf.constant(0)
    c = lambda i: tf.less(i, width)

    def loop_width(i):
        tf.print(i)
        return (tf.add(i, 1),)
    r = tf.while_loop(c, loop_width, [i])
    return r

    """
    matrix: inverse matrix, i.e., map from dest to source
    width: width of destination
    height: height of destination
    """
    # old_height, old_width, channel_num = image.shape
    # # dst = tf.Variable(tf.zeros((height, width, channel_num), dtype=tf.float32), trainable=False)
    # def do(i):
    #     tf.print(i)
    #     return (tf.add(i, 1),)
    # tf.while_loop(
    #     lambda i: tf.less(i, width),
    #     do,
    #     [tf.constant(0)]
    # )
    # for u in range(width):
    #     for v in range(height):
    #         tf.print(u)
            # x, y, _ = matrix_inv @ tf.constant([ [u], [v], [1.0] ], dtype=tf.float32)
            # x, y, = x[0], y[0]
            # intx, inty = int(x), int(y)
            # if (0 <= intx < old_width and 0 <= inty < old_height):
            #     pix = image[inty, intx]
            #     dst[v, u].assign(pix)
    # return dst

@tf.function
def rotate_image(image, degree):
    rad = degree * np.pi / 180
    height, width, _ = image.shape
    
    rotate_matrix = get_rotation_matrix(rad, height, width)
    rotate_matrix_inv = tf.linalg.inv(rotate_matrix)
    dest_img = warpAffine(image, rotate_matrix_inv, width, height)
    return dest_img

In [7]:
# tf.meshgrid(tf.range(10), tf.range(5))

In [8]:
# x = [1, 2, 3]
# y = [4, 5, 6]
# X, Y = tf.meshgrid(x, y)

In [2]:
img = Image.open('demo.jpeg')

In [3]:
img_np = np.array(img)

In [4]:
h, w, c = img_np.shape

In [5]:
rot_matrix =  get_rotation_matrix(30.0, h, w)
rot_matrix_inx = tf.linalg.inv(rot_matrix)

: 

: 